# Task

Import required libraries (for the requirements please navigate to requirements.txt in the folder)

In [1]:
import os
import requests
import re
import io
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime
import fastparquet
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
import warnings
warnings.filterwarnings("ignore")

In [6]:
# Define the data path, chart path and number of files to be downloaded
data_path = "data"
chart_path = "charts"
# Create folders if they don't exist
os.makedirs(data_path, exist_ok=True)
os.makedirs(chart_path, exist_ok=True)
# The link to the data
url = "https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/employmentandemployeetypes/timeseries/ap2y/lms/previous/"
base_url = "https://www.ons.gov.uk"
number_of_files = 1

## 1. Data Ingestion & Automation

This section parses HTML to identify and collect all relevant csv data files. For each vintage, it generates a corresponding 
vacancy_data_{releasedate}.csv file containing the extracted data. The csv files are saved under "/data" folder. While csv is a widely used format and sufficient for these relatively small datasets, it becomes less efficient as data volume grows. For larger datasets, Parquet is a better choice due to its better read/write performance, file size compression and compatibility with big data tools like Spark. I added the relevant codes to also save the data as Parquet files named as vacancy_data_{releasedate}.parquet.

### 1.1 Read Target Page Content

In [7]:
# Send request with headers to mimic a browser
headers = {
    "User-Agent": "Mozilla/5.0"
}
response = requests.get(url, headers=headers, verify=False)

### 1.2 Identify CSV Files in the Page Content 

In [8]:
# Parse HTML
soup = BeautifulSoup(response.content, "html.parser")

# Find all <a> tags with the csv download class
links = soup.find_all("a", attrs={"data-gtm-type": "download-version-csv"})

In [9]:
def download_csv_file(file_link, base_url, data_path):
    """
    Downloads a CSV file from a constructed URL and saves it locally with a filename 
    based on the release date extracted from the file content.

    Args:
        
        file_link (bs4.element.Tag): A BeautifulSoup tag object representing an <a> element, 
                                     expected to contain 'href' and 'data-gtm-date' attributes.
        base_url (str): The base URL to prepend to the 'href' to form the full download URL.
        data_path (str): The local directory path where the CSV file will be saved.

    Behavior:
        - Constructs the full URL using `base_url` and the 'href' from `file_link`.
        - Sends a GET request to download the CSV file.
        - Extracts the release date from the file content using a regular expression.
        - Formats the release date as 'YYYYMMDD' and uses it in the filename to make the files chronogically ordered.
        - Saves the file to `data_path` with the name 'vacancy_data_<release_date>.csv' and 'vacancy_data_<release_date>.parquet'.
        - If the release date is not found, defaults to 'vacancy_data_000.csv' and 'vacancy_data_000.parquet'.
        - Prints status messages indicating success or failure.

    Note:
        SSL verification is disabled in the request (`verify=False`), which may pose a security risk.
    """

    
    href = file_link.get("href")
    date_superseded = file_link.get("data-gtm-date")
    full_url = base_url + href
    
    print(f"{date_superseded}: {full_url}")

    response = requests.get(full_url, verify=False)

    if response.status_code == 200:
        # Decode content to string
        content = response.content.decode("utf-8")

        # Extract release date using regex
        match = re.search(r'"Release date","([\d\-]+)"', content)
        if match:
            release_date = match[1].replace("-", "")  # e.g., "12082025"
            parsed_date = datetime.strptime(release_date, "%d%m%Y")
            release_date = parsed_date.strftime("%Y%m%d") # Format the release date to YYYYMMDD so that downloaded files are named in chronological order
            filename_csv = os.path.join(data_path, f"vacancy_data_{release_date}.csv")
            filename_parquet = os.path.join(data_path, f"vacancy_data_{release_date}.parquet")
        else:
            filename_csv = os.path.join(data_path, f"vacancy_data_000.csv")
            filename_parquet = os.path.join(data_path, f"vacancy_data_000.parquet")

        # Save file
        with open(filename_csv, "w", encoding="utf-8") as f:
            f.write(content)

        # Save as Parquet
        df = pd.read_csv(io.StringIO(content))
        df.to_parquet(filename_parquet, engine="fastparquet", index=False)

        print(f"Downloaded and saved as '{filename_csv}'")
    else:
        print(f"Failed to download {release_date}. Status code: {response.status_code}")


In [10]:
# Download the first 24 files 
for link in links[:number_of_files]:
    download_csv_file(link, base_url, data_path)

Latest: https://www.ons.gov.uk/generator?format=csv&uri=/employmentandlabourmarket/peopleinwork/employmentandemployeetypes/timeseries/ap2y/lms
Downloaded and saved as 'data\vacancy_data_20250916.csv'


## 2. Data Cleaning & Structuring

This section brings together multiple files containing job vacancy data and combines them into one organized dataset. It starts by identifying the point (MAY 2001) in each file where consistent monthly data begins, then aligns all the data by month. Each file represents a different vintage. Once everything is aligned and sorted by date, the consolidated data is saved into the file "vacancy_data_allvintages.csv".

In [11]:
def process_vacancy_csv(file):
    """
    Processes a vacancy csv file by cleaning and transforming its contents into a time series format.

    Args:
        file (str): The csv file that will be processed.

    Returns:
        pd.DataFrame: A cleaned DataFrame with:
            - 'Date' as the index in 'YYYY-MM' format.
            - One column named after the vintage date extracted from the filename.

    Processing steps:
        - Read the csv file.
        - Cleans the 'Title' column by collapsing multiple spaces into one.
        - Converts the 'Title' column to title case and parses it as a date in 'YYYY MMM' format.
        - Drops rows where the 'Title' column could not be parsed as a date.
        - Formats the parsed dates to 'YYYY-MM'.
        - Infers a vintage identifier from the filename and renames the column accordingly.
        - Sets the 'Date' column as the index.
    """
    # Read the csv file
    df = pd.read_csv(os.path.join(data_path,file)) ##
    # Read the parquet file
    #df = pd.read_parquet(os.path.join(data_path,file))
    
    # Replace multiple spaces with one space in Title column
    df['Title'] = df['Title'].apply(lambda x: re.sub(r'\s+', ' ', x))
    # Convert Title column into date having YYYY MMM format
    df["Title"] = df["Title"].str.title()
    df["Title"] = pd.to_datetime(df["Title"], format='%Y %b', errors='coerce')
    # Drop non-date columns
    df = df.dropna(subset="Title")
    # Convert Date column into YY-MM format
    df["Title"] = df["Title"].dt.strftime("%Y-%m")
    
    # Infer the vintage date from the file name
    vintage_date = file.replace("vacancy_data_","v_").replace(".csv", "") ##
    #vintage_date = file.replace("vacancy_data_","v_").replace(".parquet", "") ##
    # Rename columns as Date and vintage date identifier
    df.columns = ["Date", vintage_date]
    # Set Date as index
    df = df.set_index("Date")

    return df

In [12]:
# List all csv/parquet files in the current directory that start with "vacancy_data"
data_files = [f for f in os.listdir(data_path) if f.startswith("vacancy_data") and f.endswith(".csv")]
#data_files = [f for f in os.listdir(data_path) if f.startswith("vacancy_data") and f.endswith(".parquet")]

In [13]:
combined_df = []
for file in data_files:
    df = process_vacancy_csv(file)
    combined_df.append(df)

In [14]:
# Merge dataframes in combined df list into a dataframe
combined_df = pd.concat(combined_df, axis=1)
combined_df = combined_df.sort_index().reset_index()
combined_df.head()

,Date,v_20230912,v_20231114,v_20231212,v_20240116,v_20240213,v_20240312,v_20240416,v_20240514,v_20240611,...,v_20241217,v_20250121,v_20250218,v_20250320,v_20250415,v_20250513,v_20250610,v_20250717,v_20250812,v_20250916
0,2001-05,680,680,680,680,680,680,680,680,680,...,680,680,680,680,680,680,680,680,680,680
1,2001-06,674,674,674,674,674,674,674,674,674,...,674,674,674,674,674,674,674,674,674,674
2,2001-07,663,663,663,663,663,663,663,663,663,...,663,663,663,663,663,663,663,663,663,663
3,2001-08,663,663,663,663,663,663,663,663,663,...,663,663,663,663,663,663,663,663,663,663
4,2001-09,639,639,639,639,639,639,639,639,639,...,639,639,639,639,639,639,639,639,639,639


In [15]:
# Save the consolidated data as a csv/parquet file
combined_df.to_csv(os.path.join(data_path, "whole_vacancy_data.csv"), index=False)
#combined_df.to_parquet(os.path.join(data_path, "whole_vacancy_data.parquet"), index=False)